In [1]:
from dash import Dash, dcc, html, Input, Output, State, ctx
from jupyter_dash import JupyterDash
import os
from IPython.display import clear_output

# ✅ Absolute assets path support
app = JupyterDash(
    __name__,
    assets_folder=os.path.abspath("../assets"),
    assets_url_path="/assets"
)

app.title = "🚗 Vehicle Carousel"

# 📦 List all subfolders in assets/object/vehicle/
VEHICLE_PATH = os.path.abspath("../assets/object/vehicle")
VEHICLE_ITEMS = sorted([
    item for item in os.listdir(VEHICLE_PATH)
    if os.path.isdir(os.path.join(VEHICLE_PATH, item))
])

# 📦 Dictionary {item: [image list]}
vehicle_images = {
    item: sorted([
        os.path.join(VEHICLE_PATH, item, img)
        for img in os.listdir(os.path.join(VEHICLE_PATH, item))
        if img.lower().endswith((".jpg", ".jpeg", ".png"))
    ])
    for item in VEHICLE_ITEMS
}

# 🎨 Layout
app.layout = html.Div([
    html.H2("🚗 Vehicle Carousel", style={
        "textAlign": "center", "marginBottom": "25px", "marginTop": "10px",
        "fontSize": "28px", "fontWeight": "bold", "fontFamily": "Georgia, serif"
    }),

    html.Div([
        html.Label("Select vehicle item:", style={"fontSize": "16px", "marginBottom": "6px", "fontFamily": "Georgia, serif"}),
        dcc.Dropdown(
            id='item-dropdown',
            options=[
                {"label": item.replace("_", " ").capitalize(), "value": item}
                for item in VEHICLE_ITEMS
            ],
            value=VEHICLE_ITEMS[0] if VEHICLE_ITEMS else None,
            clearable=False,
            style={"width": "300px", "margin": "0 auto", "fontFamily": "Georgia, serif"}
        )
    ], style={"textAlign": "center", "marginBottom": "30px"}),

    html.Div(id='carousel-container', style={"textAlign": "center"})
], style={"fontFamily": "Georgia, serif", "padding": "30px"})

# 🔁 Update carousel
@app.callback(
    Output('carousel-container', 'children'),
    Input('item-dropdown', 'value'),
    prevent_initial_call=False
)
def update_carousel(item):
    images = vehicle_images.get(item, [])
    if not images:
        return html.P("No images available.", style={"textAlign": "center", "fontSize": "18px", "color": "#e74c3c"})

    relative_path = os.path.relpath(images[0], os.path.abspath("../assets"))
    filename = os.path.basename(images[0])

    return html.Div([
        html.Div([
            html.Button("⬅️", id="prev-button", n_clicks=0, style={
                "fontSize": "18px", "padding": "5px 10px", "marginRight": "15px",
                "backgroundColor": "#f8f9fa", "border": "1px solid #ccc",
                "borderRadius": "8px", "cursor": "pointer"
            }),

            html.Img(id="carousel-image", src=f"/assets/{relative_path}", style={
                "maxHeight": "400px", "maxWidth": "400px",
                "borderRadius": "12px", "boxShadow": "0 0 6px rgba(0,0,0,0.1)"
            }),

            html.Button("➡️", id="next-button", n_clicks=0, style={
                "fontSize": "18px", "padding": "5px 10px", "marginLeft": "15px",
                "backgroundColor": "#f8f9fa", "border": "1px solid #ccc",
                "borderRadius": "8px", "cursor": "pointer"
            }),
        ], style={"display": "flex", "justifyContent": "center", "alignItems": "center"}),

        dcc.Store(id='image-index', data=0),
        dcc.Store(id='current-images', data=images),

        html.Div(f"1/{len(images)} - {filename}", id='filename-display', style={
            "fontSize": "14px", "marginTop": "10px", "color": "#444", "fontFamily": "Georgia, serif"
        })
    ])

# 🔁 Image navigation
@app.callback(
    Output("carousel-image", "src"),
    Output("image-index", "data"),
    Output("filename-display", "children"),
    Input("prev-button", "n_clicks"),
    Input("next-button", "n_clicks"),
    State("current-images", "data"),
    State("image-index", "data"),
    prevent_initial_call=True
)
def navigate_images(prev_clicks, next_clicks, images, index):
    if not images:
        return "", 0, "No images"

    triggered = ctx.triggered_id
    if triggered == "next-button":
        index = (index + 1) % len(images)
    elif triggered == "prev-button":
        index = (index - 1) % len(images)

    relative_path = os.path.relpath(images[index], os.path.abspath("../assets"))
    filename = os.path.basename(images[index])
    return f"/assets/{relative_path}", index, f"{index + 1}/{len(images)} - {filename}"

# 🚀 Run
if __name__ == "__main__":
    clear_output(wait=True)
    app.run(port=8997)
